In [8]:
import json
import numpy as np
import pandas as pd
from edgar.financials import FinancialReportEncoder
from edgar.stock import Stock
'''
https://www.pingshiuanchua.com/blog/post/intro-to-colaboratory-and-linking-it-to-google-sheets
https://www.pingshiuanchua.com/blog/post/overpower-your-google-sheets-with-python

'''

def getData(ticket, type, period='annual', year=20188, quarter=0):
    # period = 'annual' # or 'quarterly', which is the default
    # year = 2018 # can use default of 0 to get the latest
    # quarter = 1 # 1, 2, 3, 4, or default value of 0 to get the latest

    stock = Stock(ticket)
    try:
        filing = stock.get_filing(period, year, quarter)
    except:
        print('Invalid input')
        return None, False

    # financial reports (contain data for multiple years)
    if type == 'income_statements':
        statements = filing.get_income_statements()
    elif type == "balance_sheets":
        statements = filing.get_balance_sheets()
    elif type == "cash_flows":
        statements = filing.get_cash_flows()

    jsonstr = FinancialReportEncoder().encode(statements)
    data = json.loads(jsonstr)

    # print(data.keys())  # dict_keys(['company', 'date_filed', 'reports'])

    listreports = data['reports']

    columns = []
    data = []

    map_keys = []

    for report in listreports:
        # print(report.keys())  # dict_keys(['date', 'months', 'map']): string, int, dict

        for report_key in report.keys():
            if report_key == 'map':
                for map_key in report['map'].keys():
                    map_keys.append(map_key)
                    # print(map_key, report['map'][map_key].keys())  # dict_keys(['label', 'value'])
                    for key in report['map'][map_key].keys():
                        # print(key, report['map'][map_key][key])
                        if key == 'label':
                            columns.append(report['map'][map_key][key])
                        else:
                            data.append(report['map'][map_key][key])

    sec_data = pd.DataFrame([data], columns=columns)
    sec_data = sec_data.transpose()
    sec_data.reset_index(inplace=True)
    sec_data['ind'] = np.arange(len(sec_data))
    sec_data.set_index("ind", inplace=True)
    print(sec_data.head(5))
    map_data = pd.Series(map_keys)
    return sec_data, True




In [ ]:
final_df = pd.DataFrame()
cmpList = ['AAL', 'AAPL', 'ADBE', 'ADI', 'ADP', 'ADSK', 'ALGN', 'ALXN', 'AMAT', 'AMGN', 'AMZN', 'ASML', 'ATVI', 'AVGO',
           'BIDU', 'BIIB', 'BKNG', 'BMRN', 'CA', 'CDNS', 'CELG', 'CERN', 'CHKP', 'CHTR', 'CMCSA', 'COST', 'CSCO', 'CSX',
           'CTAS', 'CTRP', 'CTSH', 'CTXS', 'DISH', 'DLTR', 'EA', 'EBAY', 'ESRX', 'EXPE', 'FAST', 'FB', 'FISV', 'FOX',
           'FOXA', 'GILD', 'GOOG', 'HAS', 'HOLX', 'HSIC', 'IDXX', 'ILMN', 'INCY', 'INTC', 'INTU', 'ISRG', 'JBHT', 'JD',
           'KHC', 'KLAC', 'LBTYA', 'LBTYK', 'LRCX', 'MAR', 'MCHP', 'MDLZ', 'MELI', 'MNST', 'MSFT', 'MU', 'MXIM', 'MYL',
           'NFLX', 'NTES', 'NVDA', 'ORLY', 'PAYX', 'PCAR', 'PYPL', 'QCOM', 'QRTEA', 'REGN', 'ROST', 'SBUX', 'SHPG',
           'SIRI', 'SNPS', 'STX', 'SWKS', 'SYMC', 'TMUS', 'TSLA', 'TTWO', 'TXN', 'ULTA', 'VOD', 'VRSK', 'VRTX', 'WBA',
           'WDAY', 'WDC', 'WYNN', 'XLNX', 'XRAY']

print('Total companies', len(cmpList))

types = ['income_statements', 'balance_sheets', 'cash_flows']
years = [2019, 2018, 2017, 2016]
columns = []

for cmp in cmpList:
    cmp_df = pd.DataFrame()
    cmp_df['ind'] = np.arange(len(cmp_df))
    cmp_df.set_index("ind", inplace=True)
    for ty in types:
        writer = pd.ExcelWriter('results/results.xlsx', engine='xlsxwriter')
        for yr in years:
            typedf, valid = getData(cmp, ty, year = yr)
            if valid:
                cmp_df = pd.concat([cmp_df,typedf], axis = 1)
                columns.append(str(ty) + '_' + str(yr))
                columns.append(str(ty) + '_' + str(yr))
    
    cmp_df.columns = columns
    cmp_df.to_excel(writer, sheet_name=cmp, index = False)
    writer.save()
    writer.close()

Total companies 102
cik for AAL is 6201
getting ['10-K', '10-K/A'] filing info from https://www.sec.gov/Archives/edgar/full-index/2019/QTR4/master.idx
No annual filing info found for year=2019 quarter=4. Finding latest.
getting ['10-K', '10-K/A'] filing info from https://www.sec.gov/Archives/edgar/full-index/2019/QTR4/master.idx
getting ['10-K', '10-K/A'] filing info from https://www.sec.gov/Archives/edgar/full-index/2019/QTR3/master.idx
getting ['10-K', '10-K/A'] filing info from https://www.sec.gov/Archives/edgar/full-index/2019/QTR2/master.idx
getting ['10-K', '10-K/A'] filing info from https://www.sec.gov/Archives/edgar/full-index/2019/QTR1/master.idx
Processing SGML at https://www.sec.gov/Archives/edgar/data/6201/0000006201-19-000009.txt
could not find anything for ShortName consolidated statements of income
could not find anything for ShortName consolidated statement of earnings
could not find anything for ShortName condensed consolidated statements of income (unaudited)
could no

cik for AAL is 6201
getting ['10-K', '10-K/A'] filing info from https://www.sec.gov/Archives/edgar/full-index/2019/QTR4/master.idx
No annual filing info found for year=2019 quarter=4. Finding latest.
getting ['10-K', '10-K/A'] filing info from https://www.sec.gov/Archives/edgar/full-index/2019/QTR4/master.idx
getting ['10-K', '10-K/A'] filing info from https://www.sec.gov/Archives/edgar/full-index/2019/QTR3/master.idx
getting ['10-K', '10-K/A'] filing info from https://www.sec.gov/Archives/edgar/full-index/2019/QTR2/master.idx
getting ['10-K', '10-K/A'] filing info from https://www.sec.gov/Archives/edgar/full-index/2019/QTR1/master.idx
Processing SGML at https://www.sec.gov/Archives/edgar/data/6201/0000006201-19-000009.txt
could not find anything for ShortName consolidated statement of financial position
could not find anything for ShortName condensed consolidated statement of financial position (current period unaudited)
could not find anything for ShortName condensed consolidated sta